In [1]:
import pandas as pd
import requests
import sqlite3
import os
import time

## Conexão com o Banco de Dados

In [2]:
db_path = os.path.abspath("../data/data.db")
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

## Criação das tabelas

In [3]:
conn.execute("PRAGMA foreign_keys = ON;")

cursor.executescript(
    """
    CREATE TABLE IF NOT EXISTS deputados_56_detalhes (
        id INTEGER NOT NULL PRIMARY KEY,
        nomeCivil TEXT NOT NULL,
        cpf TEXT NOT NULL,
        dataNascimento TEXT,
        dataFalecimento TEXT,
        escolaridade TEXT,
        profissoes TEXT,
        redeSocial TEXT -- Salvar lista como uma str separada por ;
    );
    CREATE INDEX IF NOT EXISTS idx_deputados_56_detalhes_nomeCivil ON deputados_56_detalhes(id);

    CREATE TABLE IF NOT EXISTS partidos (
        id INTEGER NOT NULL PRIMARY KEY,
        sigla TEXT NOT NULL UNIQUE,
        nome TEXT NOT NULL,
        urlLogo TEXT,
        uri TEXT NOT NULL
    );

    CREATE TABLE IF NOT EXISTS deputados_56 (
        id INTEGER NOT NULL PRIMARY KEY,
        nome TEXT NOT NULL,
        siglaUf TEXT NOT NULL,
        siglaPartido TEXT NOT NULL,
        urlFoto TEXT NOT NULL,
        uri TEXT NOT NULL,
        FOREIGN KEY (id) REFERENCES deputados_56_detalhes(id) ON DELETE CASCADE
    );
    
    CREATE TABLE IF NOT EXISTS deputados_56_partidos (
        id_deputado INTEGER NOT NULL,
        siglaPartido INTEGER NOT NULL,
        "dataHora" TEXT NOT NULL,
        "situacao" TEXT NOT NULL,
        "condicaoEleitoral" TEXT NOT NULL,
        "descricaoStatus" TEXT NOT NULL,
        FOREIGN KEY (id_deputado) REFERENCES deputados_56_detalhes(id) ON DELETE CASCADE,
        FOREIGN KEY (siglaPartido) REFERENCES partidos(sigla) ON DELETE CASCADE
    );
    CREATE INDEX IF NOT EXISTS idx_deputados_56_partidos_id_deputado ON deputados_56_partidos(id_deputado);
    CREATE INDEX IF NOT EXISTS idx_deputados_56_partidos_siglaPartido ON deputados_56_partidos(siglaPartido);
    

    CREATE TABLE IF NOT EXISTS despesas (
        id_deputado INTEGER NOT NULL,
        ano INTEGER NOT NULL,
        mes INTEGER NOT NULL,
        tipoDespesa TEXT NOT NULL,
        codDocumento INTEGER NOT NULL,
        tipoDocumento TEXT NOT NULL,
        codTipoDocumento INTEGER NOT NULL,
        dataDocumento TEXT NOT NULL,
        numDocumento TEXT NOT NULL,
        valorDocumento REAL NOT NULL,
        valorLiquido REAL NOT NULL,
        urlDocumento TEXT,
        nomeFornecedor TEXT NOT NULL,
        cnpjCpfFornecedor TEXT NOT NULL,
        codLote INTEGER NOT NULL,
        FOREIGN KEY (id_deputado) REFERENCES deputados_56_detalhes(id) ON DELETE CASCADE
    );
    CREATE INDEX IF NOT EXISTS idx_despesas_deputados ON despesas(numDocumento);
    """
)

conn.commit()
conn.close()

## 1) Deputados da 56° Legislatura 2019-2022

In [4]:
base_url = "https://dadosabertos.camara.leg.br/api/v2/deputados"
params = {
    "idLegislatura": 56,
    "ordem": "ASC",
    "ordenarPor": "siglaUF"
}


with sqlite3.connect(db_path) as conn:
    all_deputados = []  
    url = base_url  

    while url:
        try:
            response = requests.get(url, params=params if url == base_url else None)
            response.raise_for_status()
            
            data = response.json()
            if "dados" in data and data["dados"]:
                df = pd.DataFrame(data["dados"])
                df = df[["id", "nome", "siglaUf", "siglaPartido", "urlFoto", "uri"]] 
                all_deputados.append(df)  


            url = None  
            if "links" in data:
                for link in data["links"]:
                    if link.get("rel") == "next":  
                        url = link.get("href")
                        break

        except requests.RequestException as e:
            print(f"Erro na requisição: {e}")
            break

        time.sleep(0.5) 


    if all_deputados:
        deputados_df = pd.concat(all_deputados, ignore_index=True)
        deputados_df.to_sql("deputados_56", conn, if_exists="replace", index=False)
        print(f"Salvos {len(deputados_df)} registros no banco.")

    else:
        print("Nenhum dado encontrado.")

Salvos 1073 registros no banco.


## 1.1) Histórico de cada partido na tabela

In [5]:
def insert_historico(conn, registro):
    sql = """INSERT OR IGNORE INTO deputados_56_partidos
             (id_deputado, siglaPartido, dataHora, situacao, condicaoEleitoral, descricaoStatus)
             VALUES (?, ?, ?, ?, ?, ?)"""
    conn.execute(sql, (
        registro["id_deputado"], registro["siglaPartido"], registro["dataHora"],
        registro["situacao"], registro["condicaoEleitoral"], registro["descricaoStatus"]
    ))
    conn.commit()

In [6]:
# Conectar ao banco de dados e buscar os IDs dos deputados da 56ª legislatura
with sqlite3.connect(db_path) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT id FROM deputados_56")  
    deputados = [row[0] for row in cursor.fetchall()]  

# Parâmetros fixos da pesquisa
base_url = "https://dadosabertos.camara.leg.br/api/v2/deputados"

for id_deputado in deputados:
    url = f"{base_url}/{id_deputado}/historico"
    params = {
        "dataInicio": "2019-01-01",
        "dataFim": "2023-12-31"
    }

    print(f"📡 Buscando histórico para deputado ID {id_deputado}...")

    while url:
        try:
            response = requests.get(url, params=params if url == f"{base_url}/{id_deputado}/historico" else None)
            response.raise_for_status()
            
            data = response.json()
            
            if "dados" in data and data["dados"]:
                with sqlite3.connect(db_path) as conn:  # Abre conexão para salvar cada registro no banco
                    for item in data["dados"]:
                        registro = {
                            "id_deputado": item["id"],
                            "siglaPartido": item["siglaPartido"],
                            "dataHora": item["dataHora"],
                            "situacao": item["situacao"],
                            "condicaoEleitoral": item["condicaoEleitoral"],
                            "descricaoStatus": item["descricaoStatus"]
                        }
                        
                        # Verifica se o registro está no intervalo de 2022
                        if 'dataHora' in registro and '2022-01-01' <= registro['dataHora'] < '2023-01-01':
                            insert_historico(conn, registro)  # Insere os dados na tabela imediatamente

            url = None  # Assume que não há mais páginas

            if "links" in data:
                for link in data["links"]:
                    if link.get("rel") == "next":  # Verifica a próxima página
                        url = link.get("href")
                        break

        except requests.RequestException as e:
            print(f"❌ Erro na requisição para o deputado {id_deputado}: {e}")
            break
        
        time.sleep(0.5)  # Pequena pausa para evitar sobrecarga da API

print("✅ Processo concluído!")

📡 Buscando histórico para deputado ID 73943...
📡 Buscando histórico para deputado ID 123756...
📡 Buscando histórico para deputado ID 141434...
📡 Buscando histórico para deputado ID 178825...
📡 Buscando histórico para deputado ID 178836...
📡 Buscando histórico para deputado ID 178839...
📡 Buscando histórico para deputado ID 204434...
📡 Buscando histórico para deputado ID 204435...
📡 Buscando histórico para deputado ID 204469...
📡 Buscando histórico para deputado ID 204471...
📡 Buscando histórico para deputado ID 204469...
📡 Buscando histórico para deputado ID 178836...
📡 Buscando histórico para deputado ID 204471...
📡 Buscando histórico para deputado ID 204471...
📡 Buscando histórico para deputado ID 204434...
📡 Buscando histórico para deputado ID 123756...
📡 Buscando histórico para deputado ID 123756...
📡 Buscando histórico para deputado ID 160541...
📡 Buscando histórico para deputado ID 171617...
📡 Buscando histórico para deputado ID 178842...
📡 Buscando histórico para deputado ID 178

## 2) Detalhes de cada Deputado

In [7]:
# Função para converter listas de redes sociais para string
def convert_rede_social(rede_social):
    if isinstance(rede_social, list):
        return '; '.join(rede_social)
    return rede_social

In [8]:
try:
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT id, uri FROM deputados_56")
        uris = cursor.fetchall()

        for id, uri in uris:
            response = requests.get(uri)
            response.raise_for_status()
            data = response.json()
                

            if "dados" in data and data["dados"]:
                dados = data["dados"]
                detalhes = {
                    "id": id,
                    "nomeCivil": dados.get("nomeCivil", ""),
                    "cpf": dados.get("cpf", ""),
                    "dataNascimento": dados.get("dataNascimento", ""),
                    "dataFalecimento": dados.get("dataFalecimento", ""),
                    "escolaridade": dados.get("escolaridade", ""),
                    "profissoes": dados.get("profissoes", ""),
                    "redeSocial": convert_rede_social(dados.get("redeSocial", ""))
                }
                
                
                
                # Verificando se o deputado já está na tabela
                cursor.execute("SELECT id FROM deputados_56_detalhes WHERE id = ?", (id,))
                result = cursor.fetchone()
                
                
                if not result:
                    cursor.execute(
                        """
                        INSERT INTO deputados_56_detalhes 
                        (id, nomeCivil, cpf, dataNascimento, dataFalecimento, escolaridade, profissoes, redeSocial)
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                        """, 
                        (
                            detalhes["id"], 
                            detalhes["nomeCivil"], 
                            detalhes["cpf"], 
                            detalhes["dataNascimento"], 
                            detalhes["dataFalecimento"], 
                            detalhes["escolaridade"], 
                            detalhes["profissoes"], 
                            detalhes["redeSocial"]
                        )
                    )
                    conn.commit()
            
                time.sleep(0.5)
                print(f"Descrição do Deputado {detalhes['id']} inserido com sucesso")

            else:
                print("Nenhum dado encontrado para a URI:", uri)

except requests.RequestException as e:
    print(f"Erro na requisição: {e}")

Descrição do Deputado 73943 inserido com sucesso
Descrição do Deputado 123756 inserido com sucesso
Descrição do Deputado 141434 inserido com sucesso
Descrição do Deputado 178825 inserido com sucesso
Descrição do Deputado 178836 inserido com sucesso
Descrição do Deputado 178839 inserido com sucesso
Descrição do Deputado 204434 inserido com sucesso
Descrição do Deputado 204435 inserido com sucesso
Descrição do Deputado 204469 inserido com sucesso
Descrição do Deputado 204471 inserido com sucesso
Descrição do Deputado 204469 inserido com sucesso
Descrição do Deputado 178836 inserido com sucesso
Descrição do Deputado 204471 inserido com sucesso
Descrição do Deputado 204471 inserido com sucesso
Descrição do Deputado 204434 inserido com sucesso
Descrição do Deputado 123756 inserido com sucesso
Descrição do Deputado 123756 inserido com sucesso
Descrição do Deputado 160541 inserido com sucesso
Descrição do Deputado 171617 inserido com sucesso
Descrição do Deputado 178842 inserido com sucesso
D

### 2.1) Inserindo profissão de cada Deputado

In [9]:
def convert_profissoes(profissoes):
    if isinstance(profissoes, list):
        # Garante que só strings válidas sejam utilizadas e ignora valores `None`
        return "; ".join(
            prof["titulo"] for prof in profissoes if "titulo" in prof and isinstance(prof["titulo"], str)
        )
    return ""

In [10]:
with sqlite3.connect(db_path) as conn:
    cursor = conn.cursor()
    
    for id, uri in uris:
        try:
            response = requests.get(
                f"https://dadosabertos.camara.leg.br/api/v2/deputados/{id}/profissoes"
            )
            response.raise_for_status()  
            data = response.json()
            
            if "dados" in data and data["dados"]:
                lista_profissoes = data["dados"]
                profissao = convert_profissoes(lista_profissoes)  
                
                cursor.execute(
                    "UPDATE deputados_56_detalhes SET profissoes = ? WHERE id = ?",
                    (profissao, id)
                )
                conn.commit()
                
                time.sleep(0.5) 
                print(f"Profissões inseridas com sucesso para ID {id}: {profissao}")
            else:
                print(f"Nenhuma profissão encontrada para ID {id}")

        except requests.RequestException as e:
            print(f"Erro na requisição para ID {id}: {e}")

Profissões inseridas com sucesso para ID 73943: Bancária; Professora
Profissões inseridas com sucesso para ID 123756: Economista
Profissões inseridas com sucesso para ID 141434: Engenheiro
Profissões inseridas com sucesso para ID 178825: Advogado; Professor
Profissões inseridas com sucesso para ID 178836: Jornalista; Administrador
Profissões inseridas com sucesso para ID 178839: Médica
Profissões inseridas com sucesso para ID 204434: Procuradora de justiça
Profissões inseridas com sucesso para ID 204435: Professor
Profissões inseridas com sucesso para ID 204469: Pastor
Profissões inseridas com sucesso para ID 204471: Jornalista; Empresária
Profissões inseridas com sucesso para ID 204469: Pastor
Profissões inseridas com sucesso para ID 178836: Jornalista; Administrador
Profissões inseridas com sucesso para ID 204471: Jornalista; Empresária
Profissões inseridas com sucesso para ID 204471: Jornalista; Empresária
Profissões inseridas com sucesso para ID 204434: Procuradora de justiça
Profi

##  4) Detalhes do partido referente ao(s) Deputado(s)

In [11]:
url = (
    "https://dadosabertos.camara.leg.br/api/v2/partidos?"
    "dataInicio=2022-01-01&"
    "dataFim=2022-12-31&"
    "ordem=ASC&"
    "ordenarPor=sigla"
)

todos_partidos = []


while url:  
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        
        for partido in data["dados"]:
            partido_info = {
                "id": partido.get("id", ""),
                "sigla": partido.get("sigla", ""),
                "nome": partido.get("nome", ""),
                "urlLogo": partido.get("urlLogo", ""),
                "uri": partido.get("uri", "")
            }
            todos_partidos.append(partido_info)

        url = None  
        for link in data["links"]: 
            if link.get("rel") == "next":  
                url = link.get("href")
                break  


    else:
        print(f"Erro na requisição ({response.status_code})")
        break


if todos_partidos:
    df_final = pd.DataFrame(todos_partidos)

    with sqlite3.connect(db_path) as conn:
        df_final.to_sql("partidos", conn, if_exists="replace", index=False)

    print("✅ Dados dos partidos armazenados com sucesso no banco de dados!")
else:
    print("❌ Nenhum dado foi coletado.")

✅ Dados dos partidos armazenados com sucesso no banco de dados!


### 4.1) Consultando uri dos partidos e atualizando urlLogo

In [12]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("SELECT id, uri FROM partidos")
partidos = cursor.fetchall()

for id, uri in partidos:
    response = requests.get(uri)
    response.raise_for_status()
    data = response.json()

    if "dados" in data and data["dados"]:
        partido = data["dados"]
        url_logo = partido.get("urlLogo", "")

        cursor.execute("UPDATE partidos SET urlLogo = ? WHERE id = ?", (url_logo, id))
        conn.commit()

        print("URL da logo atualizada com sucesso:", id)

    else:
        print("Nenhum dado encontrado para a URI:", uri)

URL da logo atualizada com sucesso: 36898
URL da logo atualizada com sucesso: 37905
URL da logo atualizada com sucesso: 36769
URL da logo atualizada com sucesso: 36899
URL da logo atualizada com sucesso: 37901
URL da logo atualizada com sucesso: 37907
URL da logo atualizada com sucesso: 36779
URL da logo atualizada com sucesso: 36786
URL da logo atualizada com sucesso: 37906
URL da logo atualizada com sucesso: 36896
URL da logo atualizada com sucesso: 37903
URL da logo atualizada com sucesso: 36813
URL da logo atualizada com sucesso: 36814
URL da logo atualizada com sucesso: 36763
URL da logo atualizada com sucesso: 36832
URL da logo atualizada com sucesso: 36833
URL da logo atualizada com sucesso: 36834
URL da logo atualizada com sucesso: 36835
URL da logo atualizada com sucesso: 36837
URL da logo atualizada com sucesso: 36839
URL da logo atualizada com sucesso: 36844
URL da logo atualizada com sucesso: 36845
URL da logo atualizada com sucesso: 36851
URL da logo atualizada com sucesso

## 5) Despesas de cada Deputado

In [13]:
with sqlite3.connect(db_path) as conn:
    deputados = pd.read_sql_query("SELECT id FROM deputados_56_detalhes", conn)

deputados_ids = deputados["id"].tolist()
despesas_nao_encontradas = []

for deputado in deputados_ids:
    url_despesas = (
        f"https://dadosabertos.camara.leg.br/api/v2/deputados/{deputado}/despesas?"
        f"idLegislatura=56&ano=2022&itens=100&ordem=ASC&ordenarPor=dataDocumento"
    )
    
    while url_despesas:  
        try:
            response = requests.get(url_despesas)
            response.raise_for_status()
            data = response.json()
            
            if "dados" in data and data["dados"]:  
                despesas = pd.DataFrame(data["dados"])

                
                if not despesas.empty:
                    despesas["id_deputado"] = deputado  # Adicionar o ID do deputado
                    despesas = despesas[[
                        "id_deputado", "ano", "mes", "tipoDespesa", "codDocumento", "tipoDocumento",
                        "codTipoDocumento", "dataDocumento", "numDocumento", "valorDocumento",
                        "valorLiquido", "urlDocumento", "nomeFornecedor", "cnpjCpfFornecedor", "codLote"
                    ]]
                    
                    # Selecionando despesas somente do ano de 2022
                    despesas = despesas[despesas["dataDocumento"].str.startswith("2022")]

        
                    with sqlite3.connect(db_path) as conn:
                        despesas.to_sql("despesas", conn, if_exists="append", index=False)
                        
                    time.sleep(0.5)  

            else:
                break


            url_despesas = None
            for link in data.get("links", []):
                if link.get("rel") == "next":
                    url_despesas = link.get("href")
                    break

        except requests.RequestException as e:
            print(f"Erro de requisição para deputado {deputado}: {e}")
            despesas_nao_encontradas.append(deputado)
            break
        except Exception as e:
            print(f"Erro inesperado ao processar deputado {deputado}: {e}")
            despesas_nao_encontradas.append(deputado)
            break


if despesas_nao_encontradas:
    print("Deputados com falha na requisição de despesas:", despesas_nao_encontradas)